In [ ]:
import pandas as pd
import re

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

pd.options.mode.chained_assignment = None 

In [ ]:
!pip install pymorphy2
import pymorphy2

In [ ]:
!pip install transformers sacremoses
from transformers import pipeline

qa_pipeline = pipeline("question-answering", "timpal0l/mdeberta-v3-base-squad2")

In [ ]:
!wget -O dataset https://www.dropbox.com/sh/d5h5f3yrql8x392/AACQ2WYa5qYCqjC8QuVZ5TJ4a?dl=1

In [ ]:
!unzip dataset

In [ ]:
!tar -xvf  '/content/task_3/full_dataset/json.tar' -C '/content/task_3'

In [ ]:
def im_pad(text):
  words = text.split()
  nominative_words = []
  for word in words:
    parsed_word = morph.parse(word)[0]
    nominative_word = parsed_word.normal_form
    nominative_words.append(nominative_word)
  nominative_sentence = " ".join(nominative_words)
  return nominative_sentence

In [ ]:
def parse_file(file_name):
  df = pd.read_json(file_name).reset_index()

  id = re.search(r'\d*', file_name).group()
  df['id'] = id

  df = df.query('index not in ("Global_deadline", "Global_supervisor")')
  df['task_text'] = df.apply(lambda x: x['Tasks']['task_text'], axis = 1)
  df['task_responsibles_people'] = df.apply(lambda x: x['Tasks']['task_responsibles_people'], axis = 1)
  df['task_responsibles_groups'] = df.apply(lambda x: x['Tasks']['task_responsibles_groups'], axis = 1)

  question = "Какая должность?"
  df['position'] = df.apply(lambda x: im_pad(qa_pipeline(question, x['task_responsibles_people'])['answer']), axis = 1)

  question = "Какое имя?"
  df['name'] = df.apply(lambda x: im_pad(qa_pipeline(question, x['task_responsibles_people'])['answer']), axis = 1)

  df = df[['id', 'index', 'task_text', 'task_responsibles_groups', 'position', 'name']]
  return df.rename(columns={'index':'task_num', 'task_responsibles_groups':'dept'})

In [ ]:
df = parse_file('/content/task_3/json/0.json')

for i in range (1, 10):
  file_name = f"/content/task_3/json/{i}.json"
  df_one = parse_file(file_name)
  df = pd.concat([df, df_one])

In [ ]:
df

,id,task_num,task_text,dept,position,name
2,,Task1,Закупить книги. Срок завершения: 19 апр 15.,Административно-хозяйственный отдел,специалист первый категория административно-хо...,кравцов и.е.
2,,Task1,Провести анализ поставщиков для замены вышедше...,Служба ИТ,специалист третий категория служба ит,ким юрий иванович
3,,Task2,Выполнить ремонт кабеля сети передачи данных м...,Служба ИТ,системный администратор,сергеев андрей
4,,Task3,Создать и настроить виртуальную машину в соотв...,Служба ИТ,специалист третий категория,ким юрий иванович
2,,Task1,Провести рефакторинг программного обеспечения ...,Отдел разработки систем,контроль над исполнение распоряжение,абаимов симон
3,,Task2,Провести рефакторинг программного обеспечения ...,Отдел разработки систем,тимлида проектный команда,абаимов симона
4,,Task3,Провести анализ накопленного технического долг...,Отдел разработки систем,программист,абашкин орест
5,,Task4,Организовать регулярное проведение статус-мити...,Отдел разработки систем,технический писатель,абалкин зураб
6,,Task5,Интегрировать в систему модуль мониторинга для...,Отдел разработки систем,специалист третий категория,андреев с.е.
7,,Task6,Интегрировать в систему модуль мониторинга для...,Отдел разработки систем,специалист третий категория отдел разработка с...,андреев с.е.
